In [2]:
!pip install transformers librosa torch ipywidgets pyspellchecker noisereduce nltk spacy
!python -m spacy download en_core_web_sm
!pip install langdetect
!pip install pydub
# !pip install sacrebleu
!pip install accelerate -U rouge.score datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

#Fine tuning


In [14]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "facebook/bart-large-cnn"

In [5]:
from datasets import load_dataset
dataset = load_dataset("samsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [6]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the "samsum" dataset
dataset = load_dataset("samsum")

# Get the training split of the dataset
# train_dataset = dataset["train"]

# # Set the desired percentage of the dataset to keep
# percentage = 0.2  # 20%

# # Split the dataset into a subset and the remaining data
# dataset, _ = train_test_split(train_dataset, train_size=percentage)

# # The "subset_dataset" variable now contains 20% of the "samsum" dataset

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
def preprocess_data(data_to_process):
  #get all the dialogues
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)

  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [9]:
# from datasets import Dataset
# dataset = Dataset.from_dict(dataset)
tokenize_data = dataset.map(preprocess_data, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [11]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [15]:
metric = load_metric('rouge')

<ipython-input-15-10e4a614ebd6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [16]:
args = Seq2SeqTrainingArguments(
    'conversation-summ', #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size= 2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    # fp16=True #available only with CUDA
    )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.289200,0.302477,41.242300,21.306800,31.823100,31.820600,60.080700
2,0.213200,0.304144,41.708900,21.521900,32.235900,32.251400,60.037900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=7366, training_loss=0.2764631950301772, metrics={'train_runtime': 4213.1865, 'train_samples_per_second': 6.993, 'train_steps_per_second': 1.748, 'total_flos': 3.1925784998313984e+16, 'train_loss': 0.2764631950301772, 'epoch': 2.0})

#Model Integration

In [ ]:
import numpy as np
import librosa
from transformers import BartForConditionalGeneration, BartTokenizer, SeamlessM4TModel, AutoProcessor
import noisereduce as nr
from google.colab import files
import time
from pydub import AudioSegment, effects
from IPython.display import display, HTML
# import sacrebleu

In [ ]:
class AudioAutoencoder(nn.Module):
    def __init__(self, input_dim, encoded_dim):
        super(AudioAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoded_dim),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_dim, input_dim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

class AudioToSummaryModel(nn.Module):
    def __init__(self, audio_feature_dim, summary_vocab_size):
        super(AudioToSummaryModel, self).__init__()
        self.fc = nn.Linear(audio_feature_dim, summary_vocab_size)

    def forward(self, x):
        summary_logits = self.fc(x)
        return summary_logits

In [ ]:
model_name_stt = "facebook/hf-seamless-m4t-medium"
model_stt = SeamlessM4TModel.from_pretrained(model_name_stt)
processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")

model_name_summarization = "facebook/bart-large-cnn"
model_summarization = BartForConditionalGeneration.from_pretrained(model_name_summarization)
tokenizer_summarization = BartTokenizer.from_pretrained(model_name_summarization)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def enhance_audio(file_path):
    try:
        audio = AudioSegment.from_file(file_path)
        normalized_audio = effects.normalize(audio)
        enhanced_file_path = "enhanced_" + file_path
        normalized_audio.export(enhanced_file_path, format="wav")
        return enhanced_file_path
    except Exception as e:
        print(f"Error enhancing audio: {e}")
        return file_path

def calculate_quality(generated_text, reference_text):
    generated_words = set(generated_text.split())
    reference_words = set(reference_text.split())
    overlap = len(generated_words.intersection(reference_words))
    total_words = len(reference_words)
    quality = (overlap / total_words) * 100 if total_words > 0 else 0
    return quality

def reduce_noise(audio, sr):
    return nr.reduce_noise(y=audio, sr=sr, n_std_thresh_stationary=1.5, prop_decrease=0.8)

def dynamic_chunk_duration(audio_length, sr=16000, max_chunks=10):
    total_duration_sec = audio_length / sr
    return int(max(30, np.ceil(total_duration_sec / max_chunks)))

def split_audio(audio_input, sr=16000, chunk_duration=30):
    chunk_length = sr * chunk_duration
    total_length = len(audio_input)
    return [audio_input[i:i + chunk_length] for i in range(0, total_length, chunk_length)]

def transcribe_audio(chunks, target_lang="eng"):
    transcriptions = []
    for chunk in chunks:
        audio_inputs = processor(audios=chunk, return_tensors="pt")
        output_tokens = model_stt.generate(**audio_inputs, tgt_lang=target_lang, generate_speech=False)
        decoded_output = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
        if decoded_output:
            transcriptions.append(decoded_output)
    return " ".join(transcriptions)

def adjust_summary_length(transcription_length):
    return (200, 50) if transcription_length > 2000 else (150, 30)

def transcribe_and_summarize_uploaded_file(file_content, filename, reference_transcript=None, reference_summary=None):
    start_time = time.time()
    try:
        with open(filename, 'wb') as f:
            f.write(file_content)

        enhanced_filename = enhance_audio(filename)

        audio_input, sr = librosa.load(enhanced_filename, sr=16000)
        audio_input = reduce_noise(audio_input, sr)
        chunk_duration = dynamic_chunk_duration(len(audio_input), sr)
        chunks = split_audio(audio_input, sr, chunk_duration)
        full_transcription = transcribe_audio(chunks)

        summary_lengths = [(100, 25), (500, 60)]
        summaries = []
        summary_qualities = []
        for max_length, min_length in summary_lengths:
            summary_ids = model_summarization.generate(
                tokenizer_summarization.encode("summarize: " + full_transcription, return_tensors="pt", max_length=1024, truncation=True),
                num_beams=4, max_length=max_length, min_length=min_length, early_stopping=True)
            summary = tokenizer_summarization.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)
            if reference_summary:
                summary_qualities.append(calculate_quality(summary, reference_summary))
            else:
                summary_qualities.append(None)

        transcript_quality = calculate_quality(full_transcription, reference_transcript) if reference_transcript else None

        processing_time = time.time() - start_time
        return full_transcription, summaries, summary_qualities, transcript_quality, processing_time
    except Exception as e:
        return f"An error occurred: {e}", [], [], None, 0

In [ ]:
def calculate_quality_bleu(generated_text, reference_text):
    bleu_score = sacrebleu.corpus_bleu([generated_text], [[reference_text]]).score
    return bleu_score

In [ ]:
def display_output(transcription, summaries, summary_qualities, processing_time, reference_transcript=None, transcript_quality=None, reference_summary=None):
    output_html = f"""
    <div style="border: 2px solid #444; border-radius: 10px; padding: 10px; margin-bottom: 10px;">
        <h3>Transcription:</h3>
        <p>{transcription}</p>
    """
    if reference_transcript:
        output_html += f"""
        <hr>
        <h3>Reference Transcript:</h3>
        <p>{reference_transcript}</p>
        """
        if transcript_quality is not None:
            output_html += f"""
            <p><strong>Transcript Quality:</strong> {transcript_quality}%</p>
            """
    if reference_summary:
        output_html += f"""
        <hr>
        <h3>Reference Summary:</h3>
        <p>{reference_summary}</p>
        """
    output_html += f"""
        <hr>
        <h3>Summaries:</h3>
        <ul>
    """
    for i, (summary, quality) in enumerate(zip(summaries, summary_qualities), start=1):
        output_html += f"""
            <li>
                <strong>Summary {i}:</strong> {summary}<br>
                """
        if quality is not None:
            output_html += f"""
                <strong>Quality:</strong> {quality}%
                """
        output_html += f"""
            </li>
        """
    output_html += f"""
        </ul>
        <hr>
        <p><strong>Processing Time:</strong> {processing_time} seconds</p>
    </div>
    """
    display(HTML(output_html))

##For demo:

In [ ]:
uploaded_files = files.upload()

for filename, file_content in uploaded_files.items():
    print(f"Processing file: {filename}")
    reference_transcript = """
# gold markets continue to look very
# strong gold has rallied rather
# significantly during the course of the
# trading session on Thursday as it looks
# like people want to have exposure
# heading into the Easter holiday weekend
# that being said any shortterm pullback
# that you get is a buying opportunity
# there will be some limited electronic
# trading uh heading into the weekend so
# do keep that in mind but it will be thin
# volume ultimately this is a market that
# I do think continues to go much higher
# but um you know whether or not we have
# the momentum to take out the shooting
# star
# from a little over a week ago then that
# remains to be seen but we certainly look
# like we're going to try to do it with
# that being said I like the idea of
# buying dips that's been the play all
# along in gold and at this point in time
# one would have to say you would
# anticipate more of the same I think that
# the
# $2,150 level is likely to continue to
# see quite a bit of support so pay
# attention to that on any pullback if we
# even get there I do think at this point
# in time it's very likely that the gold
# market could go looking to the $2500
# level although we don't necessarily have
# to get there right away gold does tend
# to be impulsive though and it could be
# much quicker than you
# think we have worked off quite a bit of
# froth from that shot higher and now it
# looks like we're ready to continue going
# higher from a longer term standpoint I
# have no interest in shorting gold quite
# frankly I think gold will probably be
# one of the better trading Vehicles this
# year as there are geopolitical concerns
# interest rates being cut and of course
# just a general concern of the politics
# and the geopolitical situation in the
# Middle East and for that matter in
# Eastern Europe if you like the video
# give me a thumbs up and make sure to
# subscribe to the channel
# """

    reference_summary = "The speaker, Chris Lewis, is bullish on gold and believes that the gold market will continue to go higher. He thinks that any short-term pullback is a buying opportunity. He also thinks that the $2,150 level is likely to continue to see quite a bit of support. His long-term target is $2,500. He believes that gold will be one of the better trading vehicles this year due to geopolitical concerns, interest rates being cut, and the general concern of the politics and the geopolitical situation in the Middle East and Eastern Europe."

    transcription, summaries, summary_qualities, transcript_quality, processing_time = transcribe_and_summarize_uploaded_file(file_content, filename, reference_transcript, reference_summary)

    display_output(transcription, summaries, summary_qualities, processing_time, reference_transcript, transcript_quality, reference_summary)


Saving PODCAST สาระนานากบสารานกรมไทยฯ  เรอง การใชยารกษาโรค.mp3 to PODCAST สาระนานากบสารานกรมไทยฯ  เรอง การใชยารกษาโรค (3).mp3
Processing file: PODCAST สาระนานากบสารานกรมไทยฯ  เรอง การใชยารกษาโรค (3).mp3


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


##For test

In [ ]:
uploaded_files = files.upload()

for filename, file_content in uploaded_files.items():
    print(f"Processing file: {filename}")
    reference_transcript = None

    reference_summary = None

    transcription, summaries, summary_qualities, transcript_quality, processing_time = transcribe_and_summarize_uploaded_file(file_content, filename, reference_transcript, reference_summary)

    display_output(transcription, summaries, summary_qualities, processing_time, reference_transcript, transcript_quality, reference_summary)


Saving PODCAST สาระนานากบสารานกรมไทยฯ  เรอง การใชยารกษาโรค.mp3 to PODCAST สาระนานากบสารานกรมไทยฯ  เรอง การใชยารกษาโรค (4).mp3
Processing file: PODCAST สาระนานากบสารานกรมไทยฯ  เรอง การใชยารกษาโรค (4).mp3


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
# transcription, summaries, summary_qualities, transcript_quality, processing_time = transcribe_and_summarize_uploaded_file(file_content, filename, reference_transcript, reference_summary)

# # Calculate BLEU scores for summaries
# summary_bleu_scores = [calculate_quality_bleu(summary, reference_summary) for summary in summaries]

# # Display BLEU scores
# for i, bleu_score in enumerate(summary_bleu_scores, start=1):
#     print(f"Summary {i} BLEU Score: {bleu_score}")
